In [ ]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import csv
import os
import numpy as np
import pandas as pd
import time
from datetime import datetime

: 

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
def calculate_angle(a,b):
    a = np.array(a)
    b = np.array(b)
    
    radians = np.arctan2(b[1]-a[1], b[0]-a[0])
    angle = np.abs(radians*180.0/np.pi)

    return angle 

In [49]:
IDEO_PATH = os.path.join('./db',os.listdir('./db')[-1])
def run(video_path):
    cap = cv2.VideoCapture(video_path)

    anomaly = {"shoulder": [], "hand": []}
    shoulder_components={"start":[],"end":[],"elapsed":[]}
    hand_components={"time":[]}

    # Initiate holistic model
    with mp_holistic.Holistic(
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    ) as holistic:
        start_time = datetime.now()
        # print(start_time)

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            # Recolor Feed
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make Detections
            results = holistic.process(image)

            # Recolor image back to BGR for rendering
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Right hand
            mp_drawing.draw_landmarks(
                image,
                results.right_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(
                    color=(80, 22, 10), thickness=2, circle_radius=4
                ),
                mp_drawing.DrawingSpec(
                    color=(80, 44, 121), thickness=2, circle_radius=2
                ),
            )

            # Left Hand
            mp_drawing.draw_landmarks(
                image,
                results.left_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(
                    color=(121, 22, 76), thickness=2, circle_radius=4
                ),
                mp_drawing.DrawingSpec(
                    color=(121, 44, 250), thickness=2, circle_radius=2
                ),
            )

            # Pose Detections
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_holistic.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(
                    color=(245, 117, 66), thickness=2, circle_radius=4
                ),
                mp_drawing.DrawingSpec(
                    color=(245, 66, 230), thickness=2, circle_radius=2
                ),
            )
            # Export coordinates
            try:
                current_time = datetime.now()
                target_time = current_time - start_time
                landmarks = results.pose_landmarks.landmark

                # Get shoulder coordinates
                left_shoulder = [
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y,
                ]
                right_shoulder = [
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y,
                ]


                # Calculate shoulder angle
                angle = calculate_angle(left_shoulder, right_shoulder)
                if angle <= 170 :  # 기준 정해야함
                    anomaly["shoulder"].append((target_time, datetime.now()))
                else:
                    shoulder_seconds = (
                        anomaly["shoulder"][-1][1] - anomaly["shoulder"][0][1]
                    ).total_seconds()
                    shoulder_anomaly_start = anomaly["shoulder"][0][0].total_seconds()
                    shoulder_anomaly_end = anomaly["shoulder"][-1][0].total_seconds()
                    if shoulder_seconds >= 1e-3:
                        shoulder_components["start"].append(shoulder_anomaly_start)
                        shoulder_components["end"].append(shoulder_anomaly_end)
                        shoulder_components["elapsed"].append(shoulder_seconds)
                    anomaly["shoulder"] = []

                if results.left_hand_landmarks or results.right_hand_landmarks:
                    hand_components["time"].append(target_time.total_seconds())

            except:
                pass

            cv2.imshow("Video Feed", image)

            if cv2.waitKey(10) & 0xFF == ord("q"):
                break

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.waitKey(1)
    cv2.waitKey(1)
    cv2.waitKey(1)
    return shoulder_components, hand_components

2023-01-17 13:47:54.703741
0.054초 동안 자세가 좋지 않았습니다.
0.218초 동안 자세가 좋지 않았습니다.
landmark {
  x: 0.8839153051376343
  y: 0.9631783366203308
  z: 4.84976624193223e-07
}
landmark {
  x: 0.8022240400314331
  y: 0.9178929328918457
  z: -0.034875351935625076
}
landmark {
  x: 0.7340267896652222
  y: 0.7931161522865295
  z: -0.05296462029218674
}
landmark {
  x: 0.6957775354385376
  y: 0.6777085065841675
  z: -0.06766431033611298
}
landmark {
  x: 0.6539362072944641
  y: 0.5998642444610596
  z: -0.08333086967468262
}
landmark {
  x: 0.7711332440376282
  y: 0.6073135137557983
  z: -0.03882317990064621
}
landmark {
  x: 0.7497815489768982
  y: 0.4506531357765198
  z: -0.07109487801790237
}
landmark {
  x: 0.7374382019042969
  y: 0.34504103660583496
  z: -0.0979555994272232
}
landmark {
  x: 0.7309594750404358
  y: 0.257902592420578
  z: -0.11685904860496521
}
landmark {
  x: 0.8301791548728943
  y: 0.5855551958084106
  z: -0.042716287076473236
}
landmark {
  x: 0.8196804523468018
  y: 0.415452539920

-1

In [23]:
shoulder_coordinates

{'left': [('0.171초', [0.7437758445739746, 0.7638124227523804]),
  ('0.243초', [0.7443177700042725, 0.7724597454071045]),
  ('0.296초', [0.7449256777763367, 0.7801184058189392]),
  ('0.351초', [0.7470946907997131, 0.7816519737243652]),
  ('0.406초', [0.7479457855224609, 0.7843669652938843]),
  ('0.460초', [0.7487296462059021, 0.7895128726959229]),
  ('0.513초', [0.7511160373687744, 0.796643853187561]),
  ('0.566초', [0.752426028251648, 0.7964356541633606]),
  ('0.622초', [0.752579927444458, 0.7986810207366943]),
  ('0.677초', [0.7548059821128845, 0.806104302406311]),
  ('0.736초', [0.7584478855133057, 0.817323625087738]),
  ('0.796초', [0.7595947980880737, 0.8177509903907776]),
  ('0.851초', [0.7601860761642456, 0.817503809928894]),
  ('0.907초', [0.7631199359893799, 0.8174516558647156]),
  ('0.963초', [0.7634538412094116, 0.8189364671707153]),
  ('1.019초', [0.7631155848503113, 0.8179270625114441]),
  ('1.074초', [0.7605997323989868, 0.7898876667022705]),
  ('1.129초', [0.7601410150527954, 0.7679232358

In [48]:
results.pose_world_landmarks

landmark {
  x: 0.03230539709329605
  y: -0.5345188975334167
  z: -0.36338454484939575
  visibility: 0.9968888163566589
}
landmark {
  x: 0.03645676001906395
  y: -0.5765987634658813
  z: -0.35107192397117615
  visibility: 0.9935089349746704
}
landmark {
  x: 0.03712974488735199
  y: -0.5774522423744202
  z: -0.35072585940361023
  visibility: 0.994691789150238
}
landmark {
  x: 0.03664396330714226
  y: -0.5778891444206238
  z: -0.3512355089187622
  visibility: 0.9926697611808777
}
landmark {
  x: 0.008692397736012936
  y: -0.5717034339904785
  z: -0.35482504963874817
  visibility: 0.994347333908081
}
landmark {
  x: 0.009322874248027802
  y: -0.5718763470649719
  z: -0.3564257323741913
  visibility: 0.9956262111663818
}
landmark {
  x: 0.008580700494349003
  y: -0.5726679563522339
  z: -0.3560633957386017
  visibility: 0.9947728514671326
}
landmark {
  x: 0.06810984015464783
  y: -0.5778087973594666
  z: -0.24307040870189667
  visibility: 0.9940974116325378
}
landmark {
  x: -0.0655939

In [26]:
results.left_hand_landmarks